In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from typing import Dict, Union
from itertools import product
import os
import xarray as xr
import hydrant.ZonalProcessing.ZonalProcessing as zp

In [7]:
# read the soil file, get it majority and save
file_path = './data/ZonalProcessing/'
file_name_csv = 'West_stats_soil_classes.csv'
df = pd.read_csv(file_path+file_name_csv)


info = {'ID': 'COMID',
        'name_of_dataset': 'soil_class',
        'explanation_of_dataset': '',
        'prefix': 'frac'}

# Mapping dictionary
mapping = {
    1: 'clay',
    2: 'silty clay',
    3: 'sandy clay',
    4: 'clay loam',
    5: 'silty clay loam',
    6: 'sandy clay loam',
    7: 'loam',
    8: 'silty loam',
    9: 'sandy loam',
    10: 'silt',
    11: 'loamy sand',
    12: 'sand'
}

ds_modified, df_modified = zp.Fraction_to_Xarray(df, info, mapping)

ds_modified

<xarray.Dataset>
Dimensions:              (COMID: 453, soil_class: 2)
Coordinates:
  * COMID                (COMID) int64 71022153 71022160 ... 71039250 71039266
  * soil_class           (soil_class) int64 3 8
Data variables:
    soil_class_majority  (COMID) int64 3 3 3 3 3 3 3 3 3 3 ... 3 3 3 3 3 3 3 3 3
    soil_class_frac      (COMID, soil_class) float64 1.0 0.0 1.0 ... 0.0 1.0 0.0
    soil_class_names     (soil_class) <U10 'sandy clay' 'silty loam'

In [17]:
# read the soil file, get it majority and save
file_path = './data/ZonalProcessing/'
file_name_csv = 'modified_West_stats_NA_NALCMS_landcover_2020_30m.csv'
df = pd.read_csv(file_path+file_name_csv)

info = {'ID': 'COMID',
        'name_of_dataset': 'land_cover_NALCMS',
        'explanation_of_dataset': '',
        'prefix': 'frac'}

LULC_name =  {1: 'Temperate or sub-polar needleleaf forest',\
              2: 'Sub-polar taiga needleleaf forest',\
              3: 'Tropical or sub-tropical broadleaf evergreen forest',\
              4: 'Tropical or sub-tropical broadleaf deciduous forest',\
              5: 'Temperate or sub-polar broadleaf deciduous forest',\
              6: 'Mixed forest',\
              7: 'Tropical or sub-tropical shrubland',\
              8: 'Temperate or sub-polar shrubland',\
              9: 'Tropical or sub-tropical grassland',\
              10: 'Temperate or sub-polar grassland',\
              11: 'Sub-polar or polar shrubland-lichen-moss',\
              12: 'Sub-polar or polar grassland-lichen-moss',\
              13: 'Sub-polar or polar barren-lichen-moss',\
              14: 'Wetland',\
              15: 'Cropland',\
              16: 'Barren lands',\
              17: 'Urban',\
              18: 'Water',\
              19: 'Snow and Ice'}


ds_modified, df_modified = zp.Fraction_to_Xarray(df, info, LULC_name)

ds_modified

<xarray.Dataset>
Dimensions:                     (COMID: 453, land_cover_NALCMS: 11)
Coordinates:
  * COMID                       (COMID) int64 71022153 71022160 ... 71039266
  * land_cover_NALCMS           (land_cover_NALCMS) int64 1 5 6 8 ... 17 18 19
Data variables:
    land_cover_NALCMS_majority  (COMID) int64 15 15 15 15 15 ... 16 16 16 1 16
    land_cover_NALCMS_frac      (COMID, land_cover_NALCMS) float64 0.0 ... 0.0
    land_cover_NALCMS_names     (land_cover_NALCMS) <U49 'Temperate or sub-po...

In [10]:
# read the soil file, get it majority and save
file_path = './data/ZonalProcessing/'
file_name_csv = 'West_stats_elv.csv'
df = pd.read_csv(file_path+file_name_csv)

info = {'ID': 'COMID'}

# Mapping dictionary
mapping = {'mean':{'units':'m'},
           'max':{'units':'m'},
           'min':{'units':'m'},
           'median':{'units':'m'}}

ds = zp.Stat_to_Xarray(df, info, mapping)
ds

<xarray.Dataset>
Dimensions:  (COMID: 453)
Coordinates:
  * COMID    (COMID) int64 71022153 71022160 71022164 ... 71039250 71039266
Data variables:
    min      (COMID) float64 849.9 859.3 884.8 ... 1.437e+03 1.395e+03 1.696e+03
    max      (COMID) float64 976.2 970.8 936.6 ... 3.117e+03 2.539e+03 3.117e+03
    mean     (COMID) float64 893.2 915.9 912.3 ... 2.161e+03 1.913e+03 2.121e+03
    median   (COMID) float64 884.6 915.8 913.4 ... 2.182e+03 1.913e+03 2.102e+03

In [20]:
# call the function to create the merger of soil and land cover
path = './data/ZonalProcessing/'
soil_type_name_csv = 'modified_West_stats_soil_classes.csv'
land_cover_name_csv = 'modified_West_stats_NA_NALCMS_landcover_2020_30m.csv'

soil_type = pd.read_csv(path+soil_type_name_csv)
land_cover = pd.read_csv(path+land_cover_name_csv)

result,report, ds =  zp.intersect_df(soil_type, land_cover,
                                     df_mappings={'df1': {'id': 'COMID', 'prefix':'frac_' , 'data_name':'soil_class'}, 
                                                  'df2': {'id': 'COMID', 'prefix':'frac_' , 'data_name':'land_cover_NALCMS'}},
                                     remove_zero_combinations = True)
ds

The indexes of all DataFrames are exactly the same with the same order.
total number of non zero combinations:  15


<xarray.Dataset>
Dimensions:            (m: 15, n: 453)
Coordinates:
  * m                  (m) object 'comb_0001' 'comb_0002' ... 'comb_0015'
  * n                  (n) int64 71022153 71022160 ... 71039250 71039266
Data variables:
    Combinations       (m) object 'soil_class3 land_cover_NALCMS1' ... 'soil_...
    soil_class         (m) object '3' '3' '3' '3' '3' ... '3' '8' '8' '8' '8'
    land_cover_NALCMS  (m) object '1' '5' '6' '8' '10' ... '1' '10' '16' '19'
    comb               (m) object 'comb_0001' 'comb_0002' ... 'comb_0015'
    fraction           (n, m) float64 0.0 0.0 0.0 0.07281 ... 0.0 0.0 0.0 0.0
    id                 (n) int64 71022153 71022160 ... 71039250 71039266